In [3]:
import tweepy
import pandas as pd
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta, timezone

Authenticate with your Twitter API credentials

-----------------------------------

In [ ]:
consumer_key = 'consumer_key'
consumer_secret = 'consumer_secret'
access_token = 'access_token'
access_token_secret = 'access_token_secret'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tweepy.API(auth)

In [ ]:
# Define search parameters
keywords = ['pollution']
exclude_retweets = True
tweets = []

# UK coordinates (latitude, longitude) and radius
uk_latitude = 54.3781
uk_longitude = -2.9376
radius = "1000km"

# Calculate date range
today = datetime.now()
one_year_ago = today - timedelta(days=365)

for keyword in keywords:
    query = f'{keyword} -filter:retweets since:{one_year_ago.strftime("%Y-%m-%d")} until:{today.strftime("%Y-%m-%d")}'
    retrieved_tweets = tweepy.Cursor(api.search_tweets, q=query, tweet_mode='extended', geocode=f"{uk_latitude},{uk_longitude},{radius}").items()
    tweets.extend(retrieved_tweets)


----------------------

pulling data from some twitter handles

-----------------------

In [ ]:
# Define search parameters
twitter_handle = '@madlendavies'  # Replace with the desired Twitter handle
num_tweets = 1000

# Define search querysewateruk
keywords = ['pollution', 'waste', 'water']
search_query = ' OR '.join(keywords)

# Define start and end dates
start_date = datetime(2021, 1, 1, 0, 0, 0, tzinfo=timezone.utc)
end_date = datetime(2023, 12, 31, 23, 59, 59, tzinfo=timezone.utc)

# Retrieve tweets
tweets = []
for tweet in tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended', lang='en').items(num_tweets):
    if not tweet.retweeted and start_date <= tweet.user.created_at <= end_date:
        tweets.append(tweet)


------------------------

structuring the unstructured data returned from the twitter api

--------------------

In [ ]:
dataset = []

for tweet in tweets:
    tweet_dict = {
        'created_at': tweet.created_at,
        'id': tweet.id,
        'text': tweet.full_text,
        'source': tweet.source,
        'truncated': tweet.truncated,
        'in_reply_to_status_id': tweet.in_reply_to_status_id,
        'in_reply_to_user_id': tweet.in_reply_to_user_id,
        'in_reply_to_screen_name': tweet.in_reply_to_screen_name,
        'user_id': tweet.user.id,
        'user_name': tweet.user.name,
        'user_screen_name': tweet.user.screen_name,
        'user_location': tweet.user.location,
        'user_description': tweet.user.description,
        'user_followers_count': tweet.user.followers_count,
        'user_friends_count': tweet.user.friends_count,
        'user_listed_count': tweet.user.listed_count,
        'user_favourites_count': tweet.user.favourites_count,
        'user_statuses_count': tweet.user.statuses_count,
        'user_created_at': tweet.user.created_at,
#         'coordinates': tweet.coordinates.coordinates if tweet.coordinates else None,
        'place': tweet.place.full_name if tweet.place else None,
        'is_quote_status': tweet.is_quote_status,
        'favorite_count': tweet.favorite_count,
        'lang': tweet.lang,
        'image_url': tweet.entities['media'][0]['media_url_https'] if 'media' in tweet.entities and tweet.entities['media'][0]['type'] == 'photo' else None
    }
    dataset.append(tweet_dict)

df = pd.DataFrame(dataset)


-----------------------------

combining the dataset from different handles together

--------------------------------

In [117]:
df = pd.read_csv('HOD/AnglianWater.csv')
df1 = pd.read_csv('HOD/nwater_care.csv')
df2 = pd.read_csv('HOD/DefraGovUK.csv')
df3 = pd.read_csv('HOD/DwrCymru.csv')
df4 = pd.read_csv('HOD/H20EU.csv')
df5 = pd.read_csv('HOD/hafrendcymru.csv')
df6 = pd.read_csv('HOD/@JNCC_UK.csv')
df7 = pd.read_csv('HOD/LDNWaterkeeper.csv')
df8 = pd.read_csv('HOD/NorthumbrianH20.csv')
df9 = pd.read_csv('HOD/@theriverstrust.csv')
df10 = pd.read_csv('HOD/sascampaigns.csv')
df11 = pd.read_csv('HOD/sewateruk.csv')
df12 = pd.read_csv('HOD/SouthernWater.csv')
df13 = pd.read_csv('HOD/SouthWestWater.csv')
df14 = pd.read_csv('HOD/thameswater.csv')
df15 = pd.read_csv('HOD/WorldBankWater.csv')

df16 = pd.read_csv('HOD/stwater.csv')
df17 = pd.read_csv('HOD/uw_wwt.csv')
df18 = pd.read_csv('HOD/UK_WIR.csv')
df19 = pd.read_csv('HOD/unitedutilities.csv')
df20 = pd.read_csv('HOD/wessexwater.csv')

In [118]:
data = pd.concat((df,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20), axis =0)

In [119]:
data = data[~data.duplicated(subset =['text', 'id'])] # removing duplicated tweets

In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6878 entries, 0 to 182
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   created_at               6878 non-null   object 
 1   id                       6878 non-null   int64  
 2   text                     6878 non-null   object 
 3   source                   6878 non-null   object 
 4   truncated                6878 non-null   bool   
 5   in_reply_to_status_id    1778 non-null   float64
 6   in_reply_to_user_id      1814 non-null   float64
 7   in_reply_to_screen_name  1814 non-null   object 
 8   user_id                  6878 non-null   int64  
 9   user_name                6877 non-null   object 
 10  user_screen_name         6878 non-null   object 
 11  user_location            3350 non-null   object 
 12  user_description         5249 non-null   object 
 13  user_followers_count     6878 non-null   int64  
 14  user_friends_count       

In [122]:
dfs = pd.read_csv('/home/c4leb/enomfon/HOD/tweet_UK/tweet_UK4k.csv')
test = pd.concat((dfs, data), axis = 0)
data = test[~test.duplicated()]
data.shape

In [126]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11050 entries, 0 to 182
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   created_at               11050 non-null  object 
 1   id                       11050 non-null  int64  
 2   text                     11050 non-null  object 
 3   source                   11050 non-null  object 
 4   truncated                11050 non-null  bool   
 5   in_reply_to_status_id    3860 non-null   float64
 6   in_reply_to_user_id      3973 non-null   float64
 7   in_reply_to_screen_name  3973 non-null   object 
 8   user_id                  11050 non-null  int64  
 9   user_name                11049 non-null  object 
 10  user_screen_name         11050 non-null  object 
 11  user_location            7499 non-null   object 
 12  user_description         9149 non-null   object 
 13  user_followers_count     11050 non-null  int64  
 14  user_friends_count      

------------------

converting the date column to datetime and filtering the tweets that are created before 2020

-------------------

In [127]:
data['user_created_at'] = pd.to_datetime(data['user_created_at'])
# Check for dates below 2021
mask = data['user_created_at'].dt.year < 2020

# Filter the DataFrame based on the mask
data = data[~mask]

In [128]:
data.lang.unique() #we have two languages here, english, french, and undefined

array(['en', 'fr', 'und'], dtype=object)

In [ ]:
other_lang = data[data.lang != 'en']

In [67]:
# data[['user_location']].to_csv('user_location.csv', index = False)

------------

Geocoding the location column to get the longitude and latitude

-------------

In [13]:
data = pd.read_csv('user_location.csv')
print(data)

geolocator = Nominatim(user_agent="myGeocoder")
# Store the market names and their corresponding coordinates in a dictionary
market_dict = {}
for market_name in data.user_location:
    try:
        location = geolocator.geocode(market_name, country_codes='GB')
        coordinates = f"({location.latitude},{location.longitude})"
        market_dict[market_name] = coordinates
    except:
        print(f"Failed to find coordinates for market: {market_name}")

df = pd.DataFrame(market_dict.items(), columns = ['name', 'cordinats'])
# df.to_csv('user_cordinates.csv', index = False)

                        user_location
0                     Wonderful Wales
1                               Paris
2                       Paris, France
3                     London, England
4                                  UK
...                               ...
11465                     wintop only
11466  Jubail  Kingdom of Saudi Arabi
11467                             NaN
11468                         kwangya
11469                             NaN

[11470 rows x 1 columns]
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Lille, France
Failed to find coordinates for market: Nenagh, Tipperary, Ireland
Failed to find coordinates for market: Ile-de-France, France
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Nederland
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Rouen, France
Failed to find coordinates for market: Brussels
Failed to find coordinates fo

Failed to find coordinates for market:  Kent Coast,  England
Failed to find coordinates for market: Nord-Pas-de-Calais
Failed to find coordinates for market: Oregon, USA
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Brussels, Belgium
Failed to find coordinates for market: Rennes
Failed to find coordinates for market: Rennes
Failed to find coordinates for market: Diksmuide, België
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Brussels
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Ile-de-France, France
Failed to find coordinates for market: PARIS France
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Brussels, Belgium
Failed to find coordinates for market: Brussels, Belgium
Failed to find coordinates for 

Failed to find coordinates for market: Versailles, France
Failed to find coordinates for market: Boulogne
Failed to find coordinates for market: Forges-les-Eaux, France
Failed to find coordinates for market: University College Cork
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Nantes, France
Failed to find coordinates for market: nan
Failed to find coordinates for market: Compiègne, France
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: douai
Failed to find coordinates for market: Brussels
Failed to find coordinates for market: Brussels, Belgium
Failed to find coordinates for market: IGNY
Failed to find coordinates for market: Rennes
Failed to find coordinates for market: Brussels, Belgium
Failed to find coordinates for market: Rennes, France
Failed to find coordinates for market: Paris - Bruxelles
Failed to find coordinates for market: Pantin, France
Failed to find coordinates for market: Paris, 

Failed to find coordinates for market: Ile-de-France, France
Failed to find coordinates for market: France Poitiers
Failed to find coordinates for market: near Norwich
Failed to find coordinates for market: Brussels, Belgium
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Dreux, France
Failed to find coordinates for market: Montreuil, France
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Rennes, Bretagne
Failed to find coordinates for market: Caen, France
Failed to find coordinates for market: chatenay malabry
Failed to find coordinates for market: Paris, Ile-de-France
Failed to find coordinates for market: Anet Eure et Loir France
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Quimper, France
Failed to find coordinates for market: nan
Failed to find coordinates for market: Bruxelles
Failed to find coordinates for market: Paris, France
Failed to find coo

Failed to find coordinates for market: Poitou-Charentes, France
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Leuven
Failed to find coordinates for market: Lille, France
Failed to find coordinates for market: Coulommiers, France
Failed to find coordinates for market: Caen, France
Failed to find coordinates for market: Poissy, France
Failed to find coordinates for market: franqueville saint pierre
Failed to find coordinates for market: Joinville-le-Pont, France
Failed to find coordinates for market: Sedan, France
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Puteaux, France
Failed to find coordinates for market: Paris - France
Failed to find coordinates for market: Seine-Saint-Denis
Failed to find coordinates for market: Val-de-Marne
Failed to find coordinates for market: Paris, France
Failed to find coordinates for market: Hauts-de-Seine
Failed to find coordinates for market: Angers
Failed to fi

...

Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: Qsmp/hermitcraft
Failed to find coordinates for market: nan
Failed to find coordinates for market: Arizona, USA
Failed to find coordinates for market: nan
Failed to find coordinates for market: My Garage
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: she/her preferred, 15
Failed to find coordinates for market: nan
Failed to find coordinates for market: 30/ They|Fae ✨Ace|Agender
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: TX.
Failed to find coordinates for market: Abuja, Nigeria
Failed to find coordinates for market: nan
Failed to find coo

Failed to find coordinates for market: Fayetteville
Failed to find coordinates for market: Lexington ky
Failed to find coordinates for market: Eau Claire, WI
Failed to find coordinates for market: nan
Failed to find coordinates for market: Winnipeg, Manitoba🇨🇦
Failed to find coordinates for market: nan
Failed to find coordinates for market: #karmalVnd
Failed to find coordinates for market: Sabah, Malaysia
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: Atlanta, GA
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: Wyoming, USA
Failed to find coordinates for market: Tennessee Oblast
Failed to find coordinates for market: Illinois, USA
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for mark

Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: 𝓼𝓸𝓶𝓮𝔀𝓱𝓮𝓻𝓮
Failed to find coordinates for market: NHS or Tories - Choose wisely.
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: ctto
Failed to find coordinates for market: n'stamp moe-moe
Failed to find coordinates for market: nan
Failed to find coordinates for market: Acche logo ke Dil Meain Aur ..
Failed to find coordinates for market: nan
Failed to find coordinates for market: Kumasi, Ghana
Failed to find coordinates for market: nan
Failed to find coordinates for market: Florida by way of Nashville
Failed to find coordinates for market: Pakistan
Failed to find coordinates for market: nan
Failed to find coordinates for market: Anger Land 
Failed to find coord

Failed to find coordinates for market: Survived in the Trenches
Failed to find coordinates for market: nan
Failed to find coordinates for market: Accra, Ghana
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: Accra, Ghana
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: Ghana
Failed to find coordinates for market: Lagos, Nigeria
Failed to find coordinates for market: Nairofey
Failed to find coordinates for market: #Paris☕️ CO•NM•AZ SãoPaulo
Failed to find coordinates for market: 𝓟𝓾𝓻𝓹𝓵𝓮 𝓞𝓬𝓮𝓪𝓷 💜
Failed to find coordinates for market: nan
Failed to find coordinates for market: Kampala, Uganda
Failed to find coordinates for market: Sacramento, CA
Failed to find coordinates for market: nan
Failed to find coordinates for market: 에스파 • 광야
Failed to find coordinates for market: nan
Failed to find coordi

Failed to find coordinates for market: Colorado, USA
Failed to find coordinates for market: Pune, India
Failed to find coordinates for market: Lancashire-over-the-sands 
Failed to find coordinates for market: Money land 
Failed to find coordinates for market: SMstan
Failed to find coordinates for market: 🌐🌎🔥
Failed to find coordinates for market: Game Academy🇳🇬
Failed to find coordinates for market: exo planet
Failed to find coordinates for market: ghostie land
Failed to find coordinates for market: any pronouns
Failed to find coordinates for market: 최수빈
Failed to find coordinates for market: nan
Failed to find coordinates for market: Pakistan
Failed to find coordinates for market: a secure location
Failed to find coordinates for market: Road to Self-Excellence 
Failed to find coordinates for market: Nigeria
Failed to find coordinates for market: nan
Failed to find coordinates for market: nan
Failed to find coordinates for market: Germany
Failed to find coordinates for market: nan
Fail

In [131]:
def extract_coordinates(df):
    df['longitude'] = df['cordinats'].apply(lambda x: float(x.split(',')[1][1:-1]))
    df['latitude'] = df['cordinats'].apply(lambda x: float(x.split(',')[0][1:]))


df = pd.read_csv('user_cordinates.csv')
extract_coordinates(df)


In [132]:
df.columns = ['user_location','cordinats', 'longitude', 'latitude']

In [133]:
df

,user_location,cordinats,longitude,latitude
0,Wonderful Wales,"(51.6247261,-3.4572937)",3.457294,51.624726
1,Paris,"(53.562752,-1.7661533)",1.766153,53.562752
2,"London, England","(51.5073359,-0.12765)",0.127650,51.507336
3,UK,"(54.7023545,-3.2765753)",3.276575,54.702354
4,East Cheshire,"(53.08951585,-2.432569348703039)",2.432569,53.089516
...,...,...,...,...
1305,Poland 🇮🇩,"(51.2726139,-0.9359351)",0.935935,51.272614
1306,Mushroom Forest,"(51.4268762,-0.7097463)",0.709746,51.426876
1307,"Brisbane, Australia","(53.3548081,-2.1577743)",2.157774,53.354808
1308,1st wife,"(51.2784665,1.0545089)",0.054509,51.278467


In [134]:
data = pd.merge(df, data, on = 'user_location', how = 'right' )

In [136]:
data.lang.unique()

array(['en', 'fr', 'und'], dtype=object)

# translation

In [95]:
api_key = 'api_key'
openai.api_key = api_key

def translate_text(text, source_language, target_language='en'):
    prompt = f"Translate the following {source_language} text to {target_language}: '{text}'"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        max_tokens=100
    )

    # Extracting the translated text from the API response
    translated_text = response['choices'][0]['message']['content']
    return translated_text.strip()


# Function to translate each row in the DataFrame
def translate_row(row):
    if row['lang'] != 'en':  # Check if the language is already English
        translated_text = translate_text(row['text'], row['lang'], 'en')
    else:
        translated_text = row['text']  # If the language is already English, keep the original text
    return translated_text

# Create a new column 'translated_text' with translated text
other_lang['translated_text'] = other_lang.apply(translate_row, axis=1)

other_lang


/tmp/ipykernel_6188/3629040288.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_lang['translated_text'] = other_lang.apply(translate_row, axis=1)


,created_at,id,text,source,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user_id,user_name,...,user_favourites_count,user_statuses_count,user_created_at,coordinates,place,is_quote_status,favorite_count,lang,image_url,translated_text
37,2023-06-24 19:12:02+00:00,1672684020583346176,"""Sain, naturel, bio"". Mise en vitrine de la su...",Twitter for Android,False,NaN,NaN,NaN,1671068817345085440,Laurence Garçon,...,26,1,2023-06-20 08:14:35+00:00,NaN,NaN,False,0,fr,https://pbs.twimg.com/media/FzaRP6hXsAQzfOx.jpg,"""Healthy, natural, organic"". Showcasing the ov..."
46,2023-06-24 18:40:31+00:00,1672676090245001216,Is #LightPollution #Pollution? @ChiArunGreen @...,Twitter Web App,False,NaN,NaN,NaN,1493183403105132545,SwitchOffTheLights,...,3082,1727,2022-02-14 11:20:49+00:00,NaN,NaN,True,1,und,NaN,Is #LightPollution #Pollution? @ChiArunGreen @...
60,2023-06-24 17:55:08+00:00,1672664667808845824,@medhi9401 @mairie15 @Paris @Space_Station @ES...,Twitter for Android,False,1.555227e+18,1.491747e+18,medhi9401,1611148050096918529,Gauthier Serrurier Paris,...,35,197,2023-01-05 23:50:24+00:00,NaN,"Drancy, France",False,0,fr,NaN,'@medhi9401 @mairie15 @Paris @Space_Station @E...
129,2023-06-24 15:06:12+00:00,1672622155471749121,@AlLouarn @LJacouille @lemondefr https://t.co/...,Twitter for Android,False,1.672594e+18,1.028613e+18,AlLouarn,1636516906021658624,Archer 🏹,...,4644,5325,2023-03-16 23:57:02+00:00,NaN,NaN,False,1,und,NaN,'@AlLouarn @LJacouille @lemondefr https://t.co...
170,2023-06-24 13:34:44+00:00,1672599136514850817,"@GeWoessner @brounno @marinetondelier Euh, il ...",Twitter for Android,False,1.672259e+18,1.222531e+08,GeWoessner,1465460222646964227,La Révolution à Vélo !,...,5113,3471,2021-11-29 23:19:01+00:00,NaN,NaN,False,1,fr,NaN,"'@GeWoessner @brounno @marinetondelier Um, whe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3985,2023-06-19 02:02:15+00:00,1670612929178116097,@JackyBerland @BonGrosDodo Cela concerne la ch...,Twitter Web App,False,1.670613e+18,1.466109e+18,AdrienBalG,1466108610707181569,AdrienBàlG ⏚,...,9420,6602,2021-12-01 18:15:45+00:00,NaN,NaN,False,1,fr,NaN,'@JackyBerland @BonGrosDodo This concerns the ...
4016,2023-06-18 22:05:34+00:00,1670553363966984196,Bah c’est super ça dis donc. Donc en plus de l...,Twitter for iPhone,False,NaN,NaN,NaN,1576260361992413191,Barbie Pecresse,...,1939,3378,2022-10-01 17:19:03+00:00,NaN,"Paris, France",True,3,fr,NaN,"Well, that's great, I must say. So, in additio..."
4093,2023-06-18 19:13:07+00:00,1670509969148354560,@pascalCenteam @f_philippot Je réponds pour ré...,Twitter for iPhone,False,1.670505e+18,8.114870e+08,pascalCenteam,1445101410660532231,Noé,...,27016,54241,2021-10-04 19:00:22+00:00,NaN,NaN,True,0,fr,NaN,"""@pascalCenteam @f_philippot I'm responding to..."
4110,2023-06-18 18:35:13+00:00,1670500430122561542,@Bruno_Attal_ Et la pollution grosse merde tou...,Twitter for Android,False,1.670438e+18,1.433049e+18,Bruno_Attal_,1640088035688607747,tony,...,182,405,2023-03-26 20:27:23+00:00,NaN,NaN,False,1,fr,NaN,"'@Bruno_Attal_ And pollution, what a big mess,..."


In [104]:
df = data.lang != 'en'
data = data[~df]
data[data.lang != 'en']

,created_at,id,text,source,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user_id,user_name,...,user_listed_count,user_favourites_count,user_statuses_count,user_created_at,coordinates,place,is_quote_status,favorite_count,lang,image_url


In [111]:
other_lang.drop(['text'], axis = 1, inplace = True)

/tmp/ipykernel_6188/1728536110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_lang.drop(['text'], axis = 1, inplace = True)


In [115]:
other_lang.rename(columns = {'translated_text': 'text'}, inplace = True)

/tmp/ipykernel_6188/1490291454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_lang.rename(columns = {'translated_text': 'text'}, inplace = True)


In [116]:
other_lang

,created_at,id,source,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user_id,user_name,user_screen_name,...,user_favourites_count,user_statuses_count,user_created_at,coordinates,place,is_quote_status,favorite_count,lang,image_url,text
37,2023-06-24 19:12:02+00:00,1672684020583346176,Twitter for Android,False,NaN,NaN,NaN,1671068817345085440,Laurence Garçon,Laurence_Garcon,...,26,1,2023-06-20 08:14:35+00:00,NaN,NaN,False,0,fr,https://pbs.twimg.com/media/FzaRP6hXsAQzfOx.jpg,"""Healthy, natural, organic"". Showcasing the ov..."
46,2023-06-24 18:40:31+00:00,1672676090245001216,Twitter Web App,False,NaN,NaN,NaN,1493183403105132545,SwitchOffTheLights,SwitchOffTheLi3,...,3082,1727,2022-02-14 11:20:49+00:00,NaN,NaN,True,1,und,NaN,Is #LightPollution #Pollution? @ChiArunGreen @...
60,2023-06-24 17:55:08+00:00,1672664667808845824,Twitter for Android,False,1.555227e+18,1.491747e+18,medhi9401,1611148050096918529,Gauthier Serrurier Paris,JeanGau16618165,...,35,197,2023-01-05 23:50:24+00:00,NaN,"Drancy, France",False,0,fr,NaN,'@medhi9401 @mairie15 @Paris @Space_Station @E...
129,2023-06-24 15:06:12+00:00,1672622155471749121,Twitter for Android,False,1.672594e+18,1.028613e+18,AlLouarn,1636516906021658624,Archer 🏹,ArcherNormandie,...,4644,5325,2023-03-16 23:57:02+00:00,NaN,NaN,False,1,und,NaN,'@AlLouarn @LJacouille @lemondefr https://t.co...
170,2023-06-24 13:34:44+00:00,1672599136514850817,Twitter for Android,False,1.672259e+18,1.222531e+08,GeWoessner,1465460222646964227,La Révolution à Vélo !,RevolutionAvelo,...,5113,3471,2021-11-29 23:19:01+00:00,NaN,NaN,False,1,fr,NaN,"'@GeWoessner @brounno @marinetondelier Um, whe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3985,2023-06-19 02:02:15+00:00,1670612929178116097,Twitter Web App,False,1.670613e+18,1.466109e+18,AdrienBalG,1466108610707181569,AdrienBàlG ⏚,AdrienBalG,...,9420,6602,2021-12-01 18:15:45+00:00,NaN,NaN,False,1,fr,NaN,'@JackyBerland @BonGrosDodo This concerns the ...
4016,2023-06-18 22:05:34+00:00,1670553363966984196,Twitter for iPhone,False,NaN,NaN,NaN,1576260361992413191,Barbie Pecresse,Drxgonz75,...,1939,3378,2022-10-01 17:19:03+00:00,NaN,"Paris, France",True,3,fr,NaN,"Well, that's great, I must say. So, in additio..."
4093,2023-06-18 19:13:07+00:00,1670509969148354560,Twitter for iPhone,False,1.670505e+18,8.114870e+08,pascalCenteam,1445101410660532231,Noé,No94559907,...,27016,54241,2021-10-04 19:00:22+00:00,NaN,NaN,True,0,fr,NaN,"""@pascalCenteam @f_philippot I'm responding to..."
4110,2023-06-18 18:35:13+00:00,1670500430122561542,Twitter for Android,False,1.670438e+18,1.433049e+18,Bruno_Attal_,1640088035688607747,tony,tony03954289794,...,182,405,2023-03-26 20:27:23+00:00,NaN,NaN,False,1,fr,NaN,"'@Bruno_Attal_ And pollution, what a big mess,..."


In [138]:
df = pd.concat((data, other_lang), axis = 0)

In [139]:
df.head(50)

,user_location,cordinats,longitude,latitude,created_at,id,text,source,truncated,in_reply_to_status_id,...,user_listed_count,user_favourites_count,user_statuses_count,user_created_at,coordinates,place,is_quote_status,favorite_count,lang,image_url
0,"London, England","(51.5073359,-0.12765)",0.127650,51.507336,2023-06-24 21:16:45+00:00,1672715409433190400,@HothfieldPlace All that pollution what “appar...,Twitter for iPhone,False,1.672699e+18,...,0,312,704,2022-12-20 15:52:12+00:00,NaN,NaN,False,0,en,NaN
1,East Cheshire,"(53.08951585,-2.432569348703039)",2.432569,53.089516,2023-06-24 21:08:08+00:00,1672713238255992834,@PetenShirl Means a lower gear and mor polluti...,Twitter Web App,False,1.671498e+18,...,1,134,3104,2022-08-31 18:58:21+00:00,NaN,NaN,False,0,en,NaN
2,"Kensington, London","(51.500841550000004,-0.17914971498845972)",0.179150,51.500842,2023-06-24 20:39:56+00:00,1672706144815415296,@toryboypierce @mailplus Londoners want ULEZ\n...,Twitter for iPhone,False,1.672705e+18,...,0,9438,4711,2022-04-07 15:05:13+00:00,NaN,NaN,False,2,en,https://pbs.twimg.com/tweet_video_thumb/FzalXs...
3,UK,"(54.7023545,-3.2765753)",3.276575,54.702354,2023-06-24 20:38:54+00:00,1672705883921326081,#LTN have reduced road space redundancy in the...,Twitter for Android,False,NaN,...,3,62642,54919,2020-10-25 15:08:34+00:00,NaN,NaN,True,3,en,NaN
4,"Birmingham, England","(52.4796992,-1.9026911)",1.902691,52.479699,2023-06-24 20:33:47+00:00,1672704594655191047,@YBcabbie @suemitch2017 @BBC @Keir_Starmer @Co...,Twitter for Android,False,1.672692e+18,...,4,34524,28163,2020-03-13 11:47:00+00:00,NaN,NaN,False,0,en,https://pbs.twimg.com/media/Fzaj9tpWIAEcwJa.jpg
5,"Bury Saint Edmunds, England","(52.2478821,0.7110913)",0.711091,52.247882,2023-06-24 19:45:57+00:00,1672692558655537152,@RonnieM9999 @IndyVoices You completely ignore...,Twitter for iPhone,False,1.672689e+18,...,4,74768,19522,2021-05-25 22:45:45+00:00,NaN,NaN,False,0,en,NaN
6,Rotherhithe london,"(51.5002908,-0.0436321)",0.043632,51.500291,2023-06-24 19:32:03+00:00,1672689060215701504,@thetastygreek @bernardn2s @WISE16 @SouthwarkL...,Twitter for iPhone,False,1.672676e+18,...,0,3324,5399,2021-06-21 13:31:30+00:00,NaN,NaN,False,0,en,NaN
7,United Kingdom,"(54.7023545,-3.2765753)",3.276575,54.702354,2023-06-24 19:22:49+00:00,1672686733756882944,@carolvorders https://t.co/2E6nBiAtxo\n\nHi @c...,Twitter for Android,False,1.672675e+18,...,0,186,465,2023-02-17 14:56:14+00:00,NaN,NaN,False,0,en,NaN
8,"Paris, France",NaN,NaN,NaN,2023-06-24 19:12:02+00:00,1672684020583346176,"""Sain, naturel, bio"". Mise en vitrine de la su...",Twitter for Android,False,NaN,...,0,26,1,2023-06-20 08:14:35+00:00,NaN,NaN,False,0,fr,https://pbs.twimg.com/media/FzaRP6hXsAQzfOx.jpg
9,"West Midlands, England","(52.5050033,-1.964396123331272)",1.964396,52.505003,2023-06-24 19:01:43+00:00,1672681427509362689,@sandwellcouncil @sandwelllabour plan to build...,Metricool,False,NaN,...,1,135,223,2021-09-03 09:52:22+00:00,NaN,NaN,False,0,en,https://pbs.twimg.com/ext_tw_video_thumb/16726...


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7828 entries, 0 to 4142
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   user_location            4291 non-null   object             
 1   cordinats                1541 non-null   object             
 2   longitude                1541 non-null   float64            
 3   latitude                 1541 non-null   float64            
 4   created_at               7828 non-null   object             
 5   id                       7828 non-null   int64              
 6   text                     7828 non-null   object             
 7   source                   7828 non-null   object             
 8   truncated                7828 non-null   bool               
 9   in_reply_to_status_id    2362 non-null   float64            
 10  in_reply_to_user_id      2418 non-null   float64            
 11  in_reply_to_screen_name  2418 

In [143]:
# df.to_csv('final_twitterData.csv', index = False)